[COVID-surge](https://github.com/dpploy/covid-surge) [https://github.com/dpploy/covid-surge] : V. F. de Almeida **14May20**

## US States Counties COVID-19 Surge Period Analysis

$
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Cmtrx}{\boldsymbol{\mathsf{C}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Qmtrx}{\boldsymbol{\mathsf{Q}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\yvec}{\boldsymbol{\mathsf{y}}}
  \newcommand{\zvec}{\boldsymbol{\mathsf{z}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \DeclareMathOperator{\rank}{rank}
  \DeclareMathOperator{\abs}{abs}
$

---
### Table of Contents<a id="toc"></a>
 - [1) Introduction.](#intro)
 - [2) Find states with fully evolved surges](#data)
 - [3) Loop over: configure run, fit model to data, analysis.](#loop)
---

## [Introduction](#toc)<a id="intro"></a>

On-line COVID-19 data is used in this notebook, and a fit to the sigmoid function 
    
\begin{equation*}
\boxed{ f(t) = \frac{\alpha_0}{1 + \alpha_1\, e^{\alpha_2\,t} } }
\end{equation*}
   
is systematically made. The time between points where the function has maximum and minimum curvature is computed and reported as the **surge period**. This period is relevant to public health officials to decide how long measures to control the epidemic should be in place. In addition, the surge period provides insight in comparing how different communities react to the epidemic.

In [ ]:
'''Load the covid-surge package'''
#!pip install --upgrade --quiet covid-surge

from covid_surge import Surge
%matplotlib inline

## [Import Data](#toc)<a id="data"></a>



In [ ]:
'''Import data'''

# Get complete US surge data
us_surge = Surge()

print('# of states/districts: ',len(us_surge.names))
print('# of days:             ',us_surge.dates.shape[0])

In [ ]:
'''Set parameters'''

us_surge.end_date = '4/20/20'   # set end date wanted
us_surge.end_date = None        # get all the data available
us_surge.ignore_last_n_days = 2 # allow for data repo to be corrected/updated
us_surge.min_n_cases_abs = 500  # min # of absolute cases for analysis
us_surge.deaths_100k_minimum = 41 # US death per 100,000 for Chronic Lower Respiratory Diseases per year: 41 (2019)

In [ ]:
'''Fit data to model function'''

# Fit data to all states
fit_data = us_surge.multi_fit_data()

states = list()

print('')
for (i,(sort_key,data)) in enumerate(fit_data):
    name = data[0]
    states.append(name)
    print('%2i) %15s: surge period %1.2f [day]'%(i,name,sort_key))

## [Loop over: Configure Run, Fit Model to Data, Analysis.](#toc)<a id="loop"></a>

In [ ]:
'''Loop over the top states and calculate the surge period for counties/towns'''

for state in states:

    print('')
    print('***************************************************************')
    print('                          ',state                               )
    print('***************************************************************')

    c_surge = Surge(locale='US',sub_locale=state)
    print('# of counties: ',len(c_surge.names))
    # Set parameters
    c_surge.end_date = '4/20/20'   # set end date wanted
    c_surge.end_date = None        # get all the data available
    c_surge.ignore_last_n_days = 2 # allow for data repo to be corrected/updated
    c_surge.min_n_cases_abs = 200  # min # of absolute cases for analysis
    c_surge.deaths_100k_minimum = 41 # US death per 100,000 for Chronic Lower Respiratory Diseases per year: 41 (2019)

    # Fit data to all counties/cities
    fit_data = c_surge.multi_fit_data()
    
    print('# of fittings done = ',len(fit_data))

    if len(fit_data) == 0:
        continue

    print('')
    for (sort_key,data) in fit_data:
        name = data[0]
        print('%15s: surge period %1.2f [day]'%(name,sort_key))

    # Create clustering bins based on surge period
    bins = c_surge.clustering(fit_data,2,'surge_period')

    print('')
    print('----------------------------------------------------------------')
    print('                            Bins                                ')
    print('----------------------------------------------------------------')
    for k in sorted(bins.keys()):
        print(' Bin %i %s'%(k,bins[k]))

    # Use bins to create groups of counties/cities based on surge period
    county_groups = dict()

    for (sort_key,data) in fit_data:
        county = data[0]
        param_vec = data[3]
        key = c_surge.get_bin_id(sort_key,bins)
        if key in county_groups:
            county_groups[key].append(county)
        else:
            county_groups[key] = list()
            county_groups[key].append(county)
                
    county_groups = [ county_groups[k] for k in
                         sorted(county_groups.keys(),reverse=False) ]

    print('')
    print('----------------------------------------------------------------')
    print('                        County Groups                           ')
    print('----------------------------------------------------------------')
    for g in county_groups:
        print(' Group %i %s'%(county_groups.index(g),g))

    # Plot the surge period for all grouped counties
    c_surge.plot_group_surge_periods( fit_data, bins )

    print('')
    print('')